In [1]:
from botasaurus_api import Api

In [34]:
import json
import csv

In [2]:
api = Api()

# Experimental Cells from here till marker below, don't need to run

In [42]:
# Create an asynchronous task
data = {
    'queries': ['Peach Garden @ The Heeren'],
    'country': None,
    'business_type': '',
    'max_cities': None,
    'randomize_cities': True,
    'api_key': '',
    'enable_reviews_extraction': True,
    'max_reviews': 50,
    'reviews_sort': 'newest',
    'lang': None,
    'max_results': None,
    'coordinates': '',
    'zoom_level': 14,
}

In [43]:
task = api.create_sync_task(data, scraper_name='google_maps_scraper')

View create_sync_task response at: ./output/responses/create_sync_task.json


In [46]:
# Fetch the task results
results = api.get_task_results(task_id=task[0]['id'])

View get_task_results response at: ./output/responses/get_task_results.json


In [32]:
featured_reviews = results['results'][0]['featured_reviews']

In [47]:
detailed_reviews = results['results'][0]['detailed_reviews']

In [53]:
results['results'][0]

{'place_id': 'ChIJq-zcK98Z2jER-biFPzPW_hs',
 'name': 'Peach Garden @ The Heeren',
 'description': '*****',
 'is_spending_on_ads': '*****',
 'reviews': 913,
 'competitors': '*****',
 'website': '*****',
 'can_claim': '*****',
 'owner': '*****',
 'featured_image': '*****',
 'main_category': 'Chinese restaurant',
 'categories': ['Chinese restaurant', 'Dim sum restaurant'],
 'rating': 4.6,
 'workday_timing': '*****',
 'is_temporarily_closed': '*****',
 'is_permanently_closed': '*****',
 'closed_on': '*****',
 'phone': '*****',
 'address': '260 Orchard Rd, #05-02 The Heeren, Singapore 238855',
 'review_keywords': '*****',
 'link': 'https://www.google.com/maps/place/Peach+Garden+@+The+Heeren/@1.3027635,103.837358,17z/data=!3m1!4b1!4m6!3m5!1s0x31da19df2bdcecab:0x1bfed6333f85b8f9!8m2!3d1.3027635!4d103.837358!16s%2Fg%2F11v6k52bg7?authuser=0&entry=ttu&g_ep=EgoyMDI0MDgyMS4wIKXMDSoASAFQAw%3D%3D',
 'status': '*****',
 'price_range': '*****',
 'reviews_per_rating': '*****',
 'featured_question': '**

In [61]:
results['results'][0]['name'].replace(' ', '_')

'Peach_Garden_@_The_Heeren'

In [63]:
f"{results['results'][0]['main_category'].split()[0]}__{results['results'][0]['name'].replace(' ', '_')}"

'Chinese__Peach_Garden_@_The_Heeren'

In [106]:
results['results'][0]['rating']

4.5

In [38]:
json_data_path = 'output/responses/get_task_results.json'

# Load the JSON data
with open(json_data_path, 'r') as file:
    data = json.load(file)

# Define the CSV file path
csv_file_path = 'output/responses/detailed_reviews.csv'

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the fieldnames for the CSV
    fieldnames = [
        'place_id', 'name', 'reviews', 'main_category', 'categories', 'rating', 'address', 'link', 'review_id', 'rating', 'review_text', 
        'published_at', 'published_at_date', 'review_likes_count', 'response_from_owner_text', 'response_from_owner_ago', 'response_from_owner_date',
        'total_number_of_reviews_by_reviewer', 'total_number_of_photos_by_reviewer', 'is_local_guide', 'review_translated_text', 'response_from_owner_translated_text'
    ]
    
    # Create a CSV DictWriter object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Process each result in the JSON data
    for result in data['results']:
        place_id = result.get('place_id')
        name = result.get('name')
        reviews = result.get('reviews')
        main_category = result.get('main_category')
        rating = result.get('rating')
        address = result.get('address')
        link = result.get('link')

        # Process each featured review
        for review in result.get('detailed_reviews', []):
            # Create a dictionary for the current review
            row = {
                'place_id': place_id,
                'name': name,
                'reviews': reviews,
                'main_category': main_category,
                'rating': rating,
                'address': address,
                'link': link,
                'review_id': review.get('review_id', ''),
                'rating': review.get('rating', ''),
                'review_text': review.get('review_text', '').replace('\n', ' ').replace('\r', ''),
                'published_at': review.get('published_at', ''),
                'published_at_date': review.get('published_at_date', ''),
                'review_likes_count': review.get('review_likes_count', ''),
                'response_from_owner_text': review.get('response_from_owner_text', ''),
                'response_from_owner_ago': review.get('response_from_owner_ago', ''),
                'response_from_owner_date': review.get('response_from_owner_date', ''),
                'total_number_of_reviews_by_reviewer': review.get('total_number_of_reviews_by_reviewer', ''),
                'total_number_of_photos_by_reviewer': review.get('total_number_of_photos_by_reviewer', ''),
                'is_local_guide': review.get('is_local_guide', ''),
                'review_translated_text': review.get('review_translated_text', ''),
                'response_from_owner_translated_text': review.get('response_from_owner_translated_text', ''),
            }
            # Write the row to the CSV file
            writer.writerow(row)

print(f"Data has been written to {csv_file_path}")


Data has been written to output/responses/detailed_reviews.csv


# Actual Code, can run cells from here on

In [98]:
def extract_names(path):
    restaurants = []
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            restaurants.append(' '.join(row).rstrip(","))
    return restaurants

In [91]:
def scrape_restaurant(restaurant):
    data = {
        'queries': [restaurant],
        'country': None,
        'business_type': '',
        'max_cities': None,
        'randomize_cities': True,
        'api_key': '',
        'enable_reviews_extraction': True,
        'max_reviews': None,
        'reviews_sort': 'newest',
        'lang': None,
        'max_results': None,
        'coordinates': '',
        'zoom_level': 14,
    }
    task = api.create_sync_task(data, scraper_name='google_maps_scraper')
    results = api.get_task_results(task_id=task[0]['id'])
    return results

In [118]:
# OLD FUNCTION, WON'T BE USED BUT KEPT FOR REFERENCE

def write_to_csv(results):
    detailed_reviews = results['results'][0]['detailed_reviews']
    file_name = f"{results['results'][0]['name'].replace(' ', '_')}"
    json_data_path = 'output/responses/get_task_results.json'
    
    # Load the JSON data
    with open(json_data_path, 'r') as file:
        data = json.load(file)
    
    # Define the CSV file path
    csv_file_path = f'output/responses/{file_name}.csv'
    
    # Open the CSV file for writing
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Define the fieldnames for the CSV
        fieldnames = [
            'place_id', 'name', 'reviews', 'main_category', 'categories', 'main_rating', 'address', 'link', 'review_id', 'rating', 'review_text', 
            'published_at', 'published_at_date', 'review_likes_count', 'response_from_owner_text', 'response_from_owner_ago', 'response_from_owner_date',
            'total_number_of_reviews_by_reviewer', 'total_number_of_photos_by_reviewer', 'is_local_guide', 'review_translated_text', 'response_from_owner_translated_text'
        ]
        
        # Create a CSV DictWriter object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Process each result in the JSON data
        for result in data['results']:
            place_id = result.get('place_id')
            name = result.get('name')
            reviews = result.get('reviews')
            main_category = result.get('main_category')
            categories = result.get('categories')
            main_rating = result.get('rating')
            address = result.get('address')
            link = result.get('link')
    
            # Process each featured review
            for review in result.get('detailed_reviews', []):
                # Create a dictionary for the current review
                row = {
                    'place_id': place_id,
                    'name': name,
                    'reviews': reviews,
                    'main_category': main_category,
                    'categories': categories,
                    'main_rating': main_rating,
                    'address': address,
                    'link': link,
                    'review_id': review.get('review_id', ''),
                    'rating': review.get('rating', ''),
                    'review_text': review.get('review_text', '').replace('\n', ' ').replace('\r', ''),
                    'published_at': review.get('published_at', ''),
                    'published_at_date': review.get('published_at_date', ''),
                    'review_likes_count': review.get('review_likes_count', ''),
                    'response_from_owner_text': review.get('response_from_owner_text', ''),
                    'response_from_owner_ago': review.get('response_from_owner_ago', ''),
                    'response_from_owner_date': review.get('response_from_owner_date', ''),
                    'total_number_of_reviews_by_reviewer': review.get('total_number_of_reviews_by_reviewer', ''),
                    'total_number_of_photos_by_reviewer': review.get('total_number_of_photos_by_reviewer', ''),
                    'is_local_guide': review.get('is_local_guide', ''),
                    'review_translated_text': review.get('review_translated_text', ''),
                    'response_from_owner_translated_text': review.get('response_from_owner_translated_text', ''),
                }
                # Write the row to the CSV file
                writer.writerow(row)
    
    print(f"Data has been written to {csv_file_path}")

In [116]:
def write_to_csvs(results):
    detailed_reviews = results['results'][0]['detailed_reviews']
    file_name = f"{results['results'][0]['name'].replace(' ', '_')}"
    json_data_path = 'output/responses/get_task_results.json'
    
    # Load the JSON data
    with open(json_data_path, 'r') as file:
        data = json.load(file)
    
    # Save restaurant details
    restaurant_file_path = f'output/responses/{file_name}.csv'
    with open(restaurant_file_path, 'w', newline='', encoding='utf-8') as csvfile:        
        restaurant_file_fieldnames = [
            'place_id', 'name', 'reviews', 'main_category', 'categories', 'main_rating', 'address', 'link'
        ]
        restaurant_writer = csv.DictWriter(csvfile, fieldnames=restaurant_file_fieldnames)
        restaurant_writer.writeheader()

        for result in data['results']:
            row = {
                'place_id': result.get('place_id'),
                'name': result.get('name'),
                'reviews': result.get('reviews'),
                'main_category': result.get('main_category'),
                'categories': result.get('categories'),
                'main_rating': result.get('rating'),
                'address': result.get('address'),
                'link': result.get('link')
            }
            restaurant_writer.writerow(row)

    # Save reviews details
    reviews_file_path = f'output/responses/{file_name}_reviews.csv'
    with open(reviews_file_path, 'w', newline='', encoding='utf-8') as csvfile:        
        reviews_file_fieldnames = [
            'place_id', 'review_id', 'rating', 'review_text', 
            'published_at', 'published_at_date', 'review_likes_count', 'response_from_owner_text', 'response_from_owner_ago', 'response_from_owner_date',
            'total_number_of_reviews_by_reviewer', 'total_number_of_photos_by_reviewer', 'is_local_guide', 'review_translated_text', 'response_from_owner_translated_text'
        ]        
        reviews_writer = csv.DictWriter(csvfile, fieldnames=reviews_file_fieldnames)        
        reviews_writer.writeheader()
        for result in data['results']:
            # Process each featured review
            for review in result.get('detailed_reviews', []):
                # Create a dictionary for the current review
                row = {
                    'place_id': result.get('place_id'),
                    'review_id': review.get('review_id', ''),
                    'rating': review.get('rating', ''),
                    'review_text': review.get('review_text', '').replace('\n', ' ').replace('\r', ''),
                    'published_at': review.get('published_at', ''),
                    'published_at_date': review.get('published_at_date', ''),
                    'review_likes_count': review.get('review_likes_count', ''),
                    'response_from_owner_text': review.get('response_from_owner_text', ''),
                    'response_from_owner_ago': review.get('response_from_owner_ago', ''),
                    'response_from_owner_date': review.get('response_from_owner_date', ''),
                    'total_number_of_reviews_by_reviewer': review.get('total_number_of_reviews_by_reviewer', ''),
                    'total_number_of_photos_by_reviewer': review.get('total_number_of_photos_by_reviewer', ''),
                    'is_local_guide': review.get('is_local_guide', ''),
                    'review_translated_text': review.get('review_translated_text', ''),
                    'response_from_owner_translated_text': review.get('response_from_owner_translated_text', ''),
                }
                reviews_writer.writerow(row)
    
    print(f"Data has been written to {restaurant_file_path}")
    print(f"Data has been written to {reviews_file_path}")

In [111]:
restaurant_list_path = 'data/restaurant_names.csv'
restaurants = extract_names(restaurant_list_path)

In [117]:
for restaurant in restaurants[13:14]:
    results = scrape_restaurant(restaurant)
    write_to_csvs(results)

View create_sync_task response at: ./output/responses/create_sync_task.json
View get_task_results response at: ./output/responses/get_task_results.json
Data has been written to output/responses/1918_Heritage_Bar_at_The_Alkaff_Mansion.csv
Data has been written to output/responses/1918_Heritage_Bar_at_The_Alkaff_Mansion_reviews.csv
